### Module 3: Explaining Models (Dictionary Learning)

In this demo, we'll use dictionary learning to analyze how the final hidden
layer of a GPTNeo model organizes articles from the fineweb-edu dataset. This
hidden layer is 768-dimensional, but analyzing individual neurons is not an
efficient way to work. We will find that looking at the learned dictionary atoms
associated with this layer's activations are much more interesting.

The libraries below link to data and models in huggingface. They are already
included in the iisa312 environment, defined in this [yaml file](https://github.com/krisrs1128/talks/blob/master/2024/20241230/examples/environment-iisa312.yaml).

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, GPTNeoModel
import torch
import numpy as np
np.random.seed(20241230)

The command below defines a data loader for the
[fineweb-edu](https://huggingface.co/datasets/HuggingFaceFW/fineweb-edu)
dataset. This is a 7.5TB dataset, so we'll only try working with a streaming
version, which allows us to read a few articles at a time (we'll be looking at a
tiny fraction of the original data, but it will be enough to see some
interesting structure).

In [2]:
fw = load_dataset("HuggingFaceFW/fineweb-edu", name="CC-MAIN-2024-10", split="train", streaming=True)

Let's save 2500 articles on which to extract activations. This might take ~ 1.5
minutes, depending on the speed of the internet connection. You can see the raw
text from a few articles below. They are all somewhat academic in style, but
they range quite dramatically in the topics they discuss.

In [3]:
n_stream = 2500
texts = []
for x in fw:
    texts.append(x["text"])
    if len(texts) > n_stream: break

texts[:10]

['– Computer viruses are parasitic programs which are able to replicate themselves, attach themselves to other executables in the computer, and perform some unwanted and often malicious actions. A virus is not able to spread itself to another computers, some user actions are needed for it to infect a new computer. Downloading and running software from untrusted sources, inserting an USB drive without a previous scan–remember always disable the AutoRun feature for the drives as CD-ROMs, DVD-ROMs– , downloading and running emails or IM attachments even from known persons, can put you in the nasty situation to have an infected computer. Always when you deal with these situations and to prevent computer infections, scan before to run.\nThe best scanners in my opinion are multi-engine online scanners like virustotal.com or novirusthanks.org. The links of these scanners and many more are on the home page.\nThe main three features of a virus are :\n– the replication mechanism search and find 

The block below extracts embeddings from the final hidden state
(`.hidden_states[-1]`) in a GPTNeo model. Notice that we're averaging the hidden dimension across all tokens in the text. In theory, we could analyze activations within smaller stretches of text, but we are aiming more for simplicity than completeness.

In [4]:
def extract_embeddings(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=False)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    return outputs.hidden_states[-1].mean(axis=(0, 1))


# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")
model = GPTNeoModel.from_pretrained("EleutherAI/gpt-neo-125m")
model = model.eval()

The block below applies `extract_embeddings` to all the articles we downloaded
above. This is relatively fast on a machine with a GPU, but since we're running
all the demos on our laptops, it would be quite slow, so I've commented it out.
Instead, we'll just download embeddings that I extracted in advance.

In [5]:
import pickle

with open('embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

The functions below define the dictionary learning algorithm. We will alternate `alpha_update` with `dictionary_update` to learn the $\alpha$ and $\Phi$ parameters, respectively. For `alpha_update`, we are using a standard implementation of fast iterative soft thresholding (this is where the seemingly arbitrary formulas like $1 + \sqrt{1 + 4t^{2}})$ are coming from). Notice that we are constraining $\alpha \succeq 0$ using the `clamp` argument of `threshold`.

In [6]:
def threshold(x, lambd, clamp=True):
    x_ = np.sign(x) * np.maximum(np.abs(x) - lambd, 0)
    if clamp:
        x_[x_ < 0] = 0
    return x_

def alpha_update(X, D, alpha, lambd, max_iter=20):
    L = np.linalg.norm(D, ord=2) ** 2
    a = np.zeros_like(alpha)
    a_ = np.copy(a)
    
    t = 1
    for _ in range(max_iter):
        a_prev = np.copy(a)
        a = threshold(a_ + (1 / L) * D.T @ (X - D @ a_), lambd / L)
        t_new = (1 + np.sqrt(1 + 4 * t ** 2)) / 2
        a_ = a + ((t - 1) / t_new) * (a - a_prev)
        t = t_new
    
    return a

def dictionary_update(X, a):
    return X @ np.linalg.pinv(a)

Let's run 25 iterations of dictionary learning with $K = 250$.

In [7]:
from tqdm import tqdm

K = 250
X = torch.stack(embeddings).numpy().T
D, N = X.shape
Phi = np.random.randn(D, K)  # Initial dictionary
alpha = np.random.rand(K, N)  # Initial sparse codes
l = 0.5  # Regularization parameter

for i in tqdm(range(25)):
    alpha = alpha_update(X, Phi, alpha, l)
    Phi = dictionary_update(X, alpha)

100%|██████████| 25/25 [00:16<00:00,  1.52it/s]


Finally, we can look at articles that have especially high activations
$\alpha_{i}$ on subsets of articles. For example, it seems the first dictionary
atom $\phi_{1}$ is mainly related to languages.

In [8]:
top_ix = np.argsort(alpha[0, :])[-10:][::-1]
[texts[i] for i in top_ix]

['Learn How to say Thank you and Sorry in Croatian\nIn any language it’s important to learn how to say thank you and sorry in Croatian. Find some common phrases in Best Languages to Learn below. Thank you in Croatian is hvala and Sorry in Croatian is Oprostite. Croatian greetings such as thank you and sorry are used in formal situations.\nGood Morning & Good Night in Croatian\nWant to wish Good Morning & Good Night in Croatian? The Croatian greetings like “Good Morning” or “Good Night” are useful to impress or catch attention of anyone around you. By using Croatian greetings, we are showing respect to others in a very efficient way. Good Morning in Croatian is dobro jutro and Good Night in Croatian is laku noć. To know more on similar greetings in other languages check Croatian vs Chinese, Croatian vs Spanish, Croatian vs French.\nBye in Croatian\nGet to know here what is bye in Croatian. Bye in Croatian language is Doviđenja. In some languages, they say bye and in some they say good-b